<a href="https://colab.research.google.com/github/manojmanivannan/ApacheSparkEssentials/blob/master/SparkStreamByPluralsight.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup Environment

There is a need to change the default python version of colab from python3.8 to python3.7.

The below cell should be run once and after it is complete , restart the runtime and run the rest (from 2nd cell onwards) of the cells

In [2]:
!echo "Python version before $(python --version)"
!apt-get update
!apt-get install -y openjdk-8-jdk-headless scala jq  ssh python3.7 > /dev/null
!if ! [[ -f "spark-2.3.1-bin-hadoop2.7.tgz" ]]; then wget -q http://archive.apache.org/dist/spark/spark-2.3.1/spark-2.3.1-bin-hadoop2.7.tgz; tar xf spark-2.3.1-bin-hadoop2.7.tgz; fi
!if ! [[ -f "ngrok-stable-linux-amd64.zip" ]]; then wget -q https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip; unzip ngrok-stable-linux-amd64.zip; fi
!if ! [[ -f "goodls_linux_amd64" ]]; then wget -q https://github.com/tanaikech/goodls/releases/download/v2.0.1/goodls_linux_amd64; chmod +x goodls_linux_amd64; fi
!cat /etc/ssh/ssh_host_rsa_key.pub > /etc/ssh/authorized_keys

#change alternatives
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.8 1
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.7 2

#check python version
!echo "Python version after $(python --version)"

# install pip for new python 
!sudo apt-get install python3.7-distutils
!wget https://bootstrap.pypa.io/get-pip.py
!python get-pip.py

# credit of these last two commands blongs to @Erik
# install colab's dependencies
!python -m pip install ipython ipython_genutils ipykernel jupyter_console prompt_toolkit httplib2 astor

# link to the old google package
!ln -s /usr/local/lib/python3.8/dist-packages/google /usr/local/lib/python3.7/dist-packages/google

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:5 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [1,567 kB]
Get:13 http://se

In [1]:
!echo "export SPARK_HOME=/content/spark-2.3.1-bin-hadoop2.7" >> /root/.bashrc
!source /root/.bashrc

In [2]:
!pip install findspark 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
# !export SPARK_HOME="/content/spark-2.3.1-bin-hadoop2.7"
# !export PATH="$PATH:$SPARK_HOME/bin"

In [9]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.3.1-bin-hadoop2.7/"

In [10]:
!/content/spark-2.3.1-bin-hadoop2.7/sbin/stop-master.sh
!/content/spark-2.3.1-bin-hadoop2.7/sbin/stop-slave.sh
!/content/spark-2.3.1-bin-hadoop2.7/sbin/start-master.sh

stopping org.apache.spark.deploy.master.Master
stopping org.apache.spark.deploy.worker.Worker
starting org.apache.spark.deploy.master.Master, logging to /content/spark-2.3.1-bin-hadoop2.7//logs/spark--org.apache.spark.deploy.master.Master-1-b57710b1187d.out


In [11]:
!sleep 3 && grep -oP "MasterUI' on port ([0-9]+)" /content/spark-2.3.1-bin-hadoop2.7/logs/spark--org.apache.spark.deploy.master*.out 
!grep -oP "spark:.*" /content/spark-2.3.1-bin-hadoop2.7/logs/spark--org.apache.spark.deploy.master*.out 

MasterUI' on port 8081
spark://b57710b1187d:7077


In [12]:
!/content/spark-2.3.1-bin-hadoop2.7/sbin/start-slave.sh $(grep -oP "spark:.*" /content/spark-2.3.1-bin-hadoop2.7/logs/spark--org.apache.spark.deploy.master*.out)
!sleep 3 && grep -oP "WorkerUI' on port ([0-9]+)" /content/spark-2.3.1-bin-hadoop2.7/logs/spark--org.apache.spark.deploy.worker*.out 

starting org.apache.spark.deploy.worker.Worker, logging to /content/spark-2.3.1-bin-hadoop2.7//logs/spark--org.apache.spark.deploy.worker.Worker-1-b57710b1187d.out
WorkerUI' on port 8082


In [13]:
get_ipython().system_raw('./ngrok authtoken 2IeJHLKYs6eB68PKJQM1VkpPu00_5JXQQZW3hofy6Q6vqTX2')
from time import sleep
def ngrok_tunnel_by_port(port: int):
  get_ipython().system_raw('pkill -f ngrok')
  get_ipython().system_raw(f'./ngrok http {port} &')
  sleep(4)
  print(get_ipython().getoutput('curl -s http://localhost:4040/api/tunnels',split=True))



Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [14]:
!jps

4001 Master
4138 Jps
4077 Worker


In [15]:
master_url = get_ipython().getoutput('grep -oP "spark:.*" /content/spark-2.3.1-bin-hadoop2.7/logs/spark--org.apache.spark.deploy.master*.out',split=True)
print('Master URL:',master_url[0])
# get_ipython().system_raw(f'nohup pyspark --master {master_url[0]} --num-executors 5 --driver-memory 4g --executor-memory 4g &' )

Master URL: spark://b57710b1187d:7077


In [16]:
!jps

4001 Master
4077 Worker
4157 Jps


In [17]:

import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf

# Spark session & context
conf = SparkConf().set("spark.master",master_url[0]).set('spark.ui.port', '4050').set("spark.driver.memory",'4g').set("spark.executor.memory",'4g')
try:
  # sc.stop()
  sc = SparkContext(conf=conf)
except ValueError:
  sc.stop()
  sc = SparkContext(conf=conf)

spark = SparkSession.builder.appName('PluralSight').getOrCreate()

2022-12-10 21:52:36 WARN  NativeCodeLoader:62 - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [18]:
spark

In [19]:
ngrok_tunnel_by_port(4050)

['{"tunnels":[{"name":"command_line","uri":"/api/tunnels/command_line","public_url":"https://2bb1-34-86-129-231.ngrok.io","proto":"https","config":{"addr":"http://localhost:4050","inspect":true},"metrics":{"conns":{"count":0,"gauge":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0},"http":{"count":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0}}},{"name":"command_line (http)","uri":"/api/tunnels/command_line%20%28http%29","public_url":"http://2bb1-34-86-129-231.ngrok.io","proto":"http","config":{"addr":"http://localhost:4050","inspect":true},"metrics":{"conns":{"count":0,"gauge":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0},"http":{"count":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0}}}],"uri":"/api/tunnels"}']


In [20]:
happiness_df = spark.read.format('csv').option('header','true').load('2016.csv')

In [21]:
happiness_df.show(5)

+-----------+--------------+--------------+---------------+-------------------------+-------------------------+------------------------+-------+------------------------+-------+-----------------------------+----------+-----------------+
|    Country|        Region|Happiness Rank|Happiness Score|Lower Confidence Interval|Upper Confidence Interval|Economy (GDP per Capita)| Family|Health (Life Expectancy)|Freedom|Trust (Government Corruption)|Generosity|Dystopia Residual|
+-----------+--------------+--------------+---------------+-------------------------+-------------------------+------------------------+-------+------------------------+-------+-----------------------------+----------+-----------------+
|    Denmark|Western Europe|             1|          7.526|                     7.46|                    7.592|                 1.44178|1.16374|                 0.79504|0.57941|                      0.44453|   0.36171|          2.73939|
|Switzerland|Western Europe|             2|         

In [22]:
happiness_df.select('Country','Region','Happiness Rank').show()

+-------------+--------------------+--------------+
|      Country|              Region|Happiness Rank|
+-------------+--------------------+--------------+
|      Denmark|      Western Europe|             1|
|  Switzerland|      Western Europe|             2|
|      Iceland|      Western Europe|             3|
|       Norway|      Western Europe|             4|
|      Finland|      Western Europe|             5|
|       Canada|       North America|             6|
|  Netherlands|      Western Europe|             7|
|  New Zealand|Australia and New...|             8|
|    Australia|Australia and New...|             9|
|       Sweden|      Western Europe|            10|
|       Israel|Middle East and N...|            11|
|      Austria|      Western Europe|            12|
|United States|       North America|            13|
|   Costa Rica|Latin America and...|            14|
|  Puerto Rico|Latin America and...|            15|
|      Germany|      Western Europe|            16|
|       Braz

In [23]:
happiness_df.select('Country','Region','Happiness Score')\
            .groupBy('Country')\
            .agg({'Happiness Score':'avg'})\
            .show(5)

+--------+--------------------+
| Country|avg(Happiness Score)|
+--------+--------------------+
|    Chad|               3.763|
|  Russia|               5.856|
|Paraguay|               5.538|
|   Yemen|               3.724|
| Senegal|               4.219|
+--------+--------------------+
only showing top 5 rows



In [24]:
sc.stop()

# High Availability setup using ZOOKEEPER

In [30]:
!echo "export SPARK_DAEMON_JAVA_OPTS=\"-Dspark.deploy.recoveryMode=ZOOKEEPER -Dspark.deploy.zookeeper.url=localhost:2181\"" > $SPARK_HOME/conf/ha.conf
!cat $SPARK_HOME/conf/ha.conf

export SPARK_DAEMON_JAVA_OPTS="-Dspark.deploy.recoveryMode=ZOOKEEPER -Dspark.deploy.zookeeper.url=localhost:2181"


In [31]:
!cp $SPARK_HOME/sbin/start-master{,-2}.sh

In [34]:
!$SPARK_HOME/sbin/stop-master.sh
!$SPARK_HOME/sbin/start-master.sh -h localhost -p 7077 --webui-port 8081 --properties-file  $SPARK_HOME/conf/ha.conf

stopping org.apache.spark.deploy.master.Master
starting org.apache.spark.deploy.master.Master, logging to /content/spark-2.3.1-bin-hadoop2.7//logs/spark--org.apache.spark.deploy.master.Master-1-b57710b1187d.out


In [37]:
!grep 'CLASS 1' $SPARK_HOME/sbin/start-master-2.sh
!sed -i -e 's/CLASS 1/CLASS 2/' $SPARK_HOME/sbin/start-master-2.sh

"${SPARK_HOME}/sbin"/spark-daemon.sh start $CLASS 1 \


In [39]:
!$SPARK_HOME/sbin/start-master-2.sh -h localhost -p 17077 --webui-port 18081 --properties-file  $SPARK_HOME/conf/ha.conf

starting org.apache.spark.deploy.master.Master, logging to /content/spark-2.3.1-bin-hadoop2.7//logs/spark--org.apache.spark.deploy.master.Master-2-b57710b1187d.out


In [40]:
!jps -lm

4705 org.apache.spark.deploy.master.Master --host b57710b1187d --port 7077 --webui-port 8080 -h localhost -p 17077 --webui-port 18081 --properties-file /content/spark-2.3.1-bin-hadoop2.7//conf/ha.conf
4595 org.apache.spark.deploy.master.Master --host b57710b1187d --port 7077 --webui-port 8080 -h localhost -p 7077 --webui-port 8081 --properties-file /content/spark-2.3.1-bin-hadoop2.7//conf/ha.conf
4772 jdk.jcmd/sun.tools.jps.Jps -lm
4077 org.apache.spark.deploy.worker.Worker --webui-port 8081 spark://b57710b1187d:7077
4175 org.apache.spark.deploy.SparkSubmit --conf spark.executor.memory=4g --conf spark.master=spark://b57710b1187d:7077 --conf spark.driver.memory=4g --conf spark.ui.port=4050 pyspark-shell


In [44]:
!grep -oP "spark:.*" $SPARK_HOME/logs/spark--org.apache.spark.deploy.master*.out

/content/spark-2.3.1-bin-hadoop2.7/logs/spark--org.apache.spark.deploy.master.Master-1-b57710b1187d.out:spark://localhost:7077
/content/spark-2.3.1-bin-hadoop2.7/logs/spark--org.apache.spark.deploy.master.Master-2-b57710b1187d.out:spark://localhost:17077


In [49]:
!$SPARK_HOME/sbin/stop-slave.sh
!$SPARK_HOME/sbin/start-slave.sh --webui-port 8082 spark://localhost:7077,localhost:17077

no org.apache.spark.deploy.worker.Worker to stop
starting org.apache.spark.deploy.worker.Worker, logging to /content/spark-2.3.1-bin-hadoop2.7//logs/spark--org.apache.spark.deploy.worker.Worker-1-b57710b1187d.out


In [50]:
ngrok_tunnel_by_port(18081)

['{"tunnels":[{"name":"command_line","uri":"/api/tunnels/command_line","public_url":"https://2f7b-34-86-129-231.ngrok.io","proto":"https","config":{"addr":"http://localhost:18081","inspect":true},"metrics":{"conns":{"count":0,"gauge":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0},"http":{"count":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0}}},{"name":"command_line (http)","uri":"/api/tunnels/command_line%20%28http%29","public_url":"http://2f7b-34-86-129-231.ngrok.io","proto":"http","config":{"addr":"http://localhost:18081","inspect":true},"metrics":{"conns":{"count":0,"gauge":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0},"http":{"count":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0}}}],"uri":"/api/tunnels"}']


In [51]:

import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf

# Spark session & context
conf = SparkConf().set("spark.master","spark://localhost:7077,localhost:17077").set('spark.ui.port', '4050').set("spark.driver.memory",'4g').set("spark.executor.memory",'4g')
try:
  # sc.stop()
  sc = SparkContext(conf=conf)
except ValueError:
  sc.stop()
  sc = SparkContext(conf=conf)

spark = SparkSession.builder.appName('PluralSight').getOrCreate()

In [52]:
ngrok_tunnel_by_port(8081)

['{"tunnels":[{"name":"command_line","uri":"/api/tunnels/command_line","public_url":"https://8d09-34-86-129-231.ngrok.io","proto":"https","config":{"addr":"http://localhost:8081","inspect":true},"metrics":{"conns":{"count":0,"gauge":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0},"http":{"count":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0}}},{"name":"command_line (http)","uri":"/api/tunnels/command_line%20%28http%29","public_url":"http://8d09-34-86-129-231.ngrok.io","proto":"http","config":{"addr":"http://localhost:8081","inspect":true},"metrics":{"conns":{"count":0,"gauge":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0},"http":{"count":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0}}}],"uri":"/api/tunnels"}']


In [53]:
happiness_df = spark.read.format('csv').option('header','true').load('2016.csv')

In [54]:
!jps

4705 Master
4595 Master
5157 CoarseGrainedExecutorBackend
5017 Worker
5228 Jps
4175 SparkSubmit


In [55]:
# kill one of the master
!kill -9 4705 

2022-12-10 22:25:34 ERROR TaskSchedulerImpl:70 - Lost executor 0 on 172.28.0.12: java.io.IOException: Failed to create directory /content/spark-2.3.1-bin-hadoop2.7/work/app-20221210222245-0000/0


In [56]:
happiness_df.select('Country','Region','Happiness Rank').show()

[Stage 1:>                                                          (0 + 1) / 1]

+-------------+--------------------+--------------+
|      Country|              Region|Happiness Rank|
+-------------+--------------------+--------------+
|      Denmark|      Western Europe|             1|
|  Switzerland|      Western Europe|             2|
|      Iceland|      Western Europe|             3|
|       Norway|      Western Europe|             4|
|      Finland|      Western Europe|             5|
|       Canada|       North America|             6|
|  Netherlands|      Western Europe|             7|
|  New Zealand|Australia and New...|             8|
|    Australia|Australia and New...|             9|
|       Sweden|      Western Europe|            10|
|       Israel|Middle East and N...|            11|
|      Austria|      Western Europe|            12|
|United States|       North America|            13|
|   Costa Rica|Latin America and...|            14|
|  Puerto Rico|Latin America and...|            15|
|      Germany|      Western Europe|            16|
|       Braz